In [3]:
from glob import glob
import pandas as pd

doc_folders = list(sorted(glob('../data/google/*/')))

doc_folders

# Let's try with 2

In [4]:
ocrs = list(sorted(glob('../data/google/*/*')))

In [5]:
import numpy as np

In [6]:
df = pd.read_csv(ocrs[0])

In [298]:
def fix_gramma(text):
    
    def o_rule(t):
        return t.replace('ө', 'о')
    
    def lower(t):
        return t.lower()
    def some_rule(t):
        return t
    fs = [o_rule, some_rule, lower]

    def apply(x):
        for f in fs:
            x = f(x)
        return x
    return [apply(t) for t in text]


In [339]:
NEAR_CONST = 15
def near(y1, y2):
    if np.abs(y1-y2) < NEAR_CONST:
        return True
    return False

def similar(seq1, seq2):
    return difflib.SequenceMatcher(a=seq1.lower(), b=seq2.lower()).ratio() > SIMILARITY_CONST


def check_text(list_of_substr, body):
    if isinstance(list_of_substr, str):
        list_of_substr = [list_of_substr.lower()]
    else:
        list_of_substr = [l.lower() for l in list_of_substr]
    for b in body:
        for substr in list_of_substr:
            if substr in b:
                return True
    for b in body:
        for substr in list_of_substr:
            if similar(substr, b):
                return True
            
    return False


def make_rows(data):
    data = data.sort_values(['upper_left_y', 'upper_left_x']).reset_index(drop=1)

    data_post = list(enumerate(data.itertuples()))
    result_rows = []
    for pos, d in data_post:
        _, prev_post = data_post[pos-1]
        if near(prev_post.upper_left_y, d.upper_left_y):
            result_rows[-1] += [d]
        else:
            result_rows += [[d]]
    
    # let's go through
    result_text = []
    for i, r in enumerate(result_rows):
        result_rows[i] = list(sorted(r, key=lambda x: x.upper_left_x))
        result_text.append(' '.join([t.text for t in result_rows[i]]))       
            
    return result_rows, result_text

In [130]:
for o in ocrs:
    df = pd.read_csv(o)
    try:
        txt = fix_gramma(make_rows(df))
        if not check_text('РАЗРЕШЕНИЕ НА ВВОД ОБЪЕКТА В ЭКСПЛУАТАЦИЮ' , txt):
            print(f'!!! {o}')
            break
    except:
        print(len(df), o)

!!! ../data/google/3_Разр. на ввод/10_0_g_ocr.csv


In [91]:
import os

In [196]:
import difflib

In [202]:
def similar(seq1, seq2):
    return difflib.SequenceMatcher(a=seq1.lower(), b=seq2.lower()).ratio() > 0.9

In [201]:
similar('ВВОД ОБЪЕКТА В ЭКСПЛУАТАЦИЮ', 'НА ВВОД ОБЪЕКТА В.ЭКСПЛУАТАЦИЮ')

0.9122807017543859


True

In [286]:
def check_doc(f):
    df = pd.read_csv(f)
    try:
        txt = fix_gramma(make_rows(df))
        if check_text('НА ВВОД ОБЪЕКТА В ЭКСПЛУАТАЦИЮ' , txt): # 3
            return 'РАЗРЕШЕНИЕ НА ВВОД ОБЪЕКТА В ЭКСПЛУАТАЦИЮ'
        elif check_text('ЭКСПЛИКАЦИЯ', txt): # 1 (2)
             return 'ЭКСПЛИКАЦИЯ'
        elif check_text('ТЕХНИЧЕСКИЙ ПАСПОРТ', txt): # 1 (1)
            return 'ТЕХНИЧЕСКИЙ ПАСПОРТ'
        elif check_text(['ДОГОВОР АРЕНДЫ ЗЕМЕЛЬНОГО УЧАСТКА', 'договор аренды земли'], txt): # 2
            return 'ДОГОВОР АРЕНДЫ ЗЕМЕЛЬНОГО УЧАСТКА'
        elif check_text(['разрешения на строительство', ], txt): # 4
            return 'РАЗРЕШЕНИЕ НА СТРОИТЕЛЬСТВО'
        elif check_text(['АРХИТЕКТУРНО-ГРАДОСТРОИТЕЛЬНОГО РЕШЕНИЯ', 
                         'АРХИТЕКТУРНО-ГРАДОСТРОИТЕЛЬНОЕ РЕШЕНИЕ'], txt): # 5
            return 'СВИДЕТЕЛЬСТВО ОБ УТВЕРЖДЕНИИ АРХИТЕКТУРНО-ГРАДОСТРОИТЕЛЬНОГО РЕШЕНИЯ'
    except:
        return None
    return None

In [272]:
fns = glob('../data/google/2_ЗУ/M_01_005599_05_06_1996 исправленное_*')

In [ ]:
fns

In [308]:

df = pd.read_csv( '../data/google/4_Разр. на стр-во/10_0_g_ocr.csv',
)
txt = fix_gramma(make_rows(df))

check_text('НА ВВОД ОБЪЕКТА В ЭКСПЛУАТАЦИЮ' , txt)

False

In [ ]:
txt

In [ ]:
from doc.clas

In [126]:
def classify_docs(files):
    fnames = set()
    for f in files:
        fnames.add(os.path.dirname(f) + '/' + os.path.basename(f).split('_')[0])
    docs = dict.fromkeys(fnames)
    for f in files:
        f_name = os.path.dirname(f) + '/' + os.path.basename(f).split('_')[0]
        docs[f_name] = check_doc(f) if (check_doc(f) is not None) else docs[f_name]
    return docs

In [328]:
def gather_docs(files):
    from collections import defaultdict

    fnames = set()
    for f in files:
        fnames.add(os.path.dirname(f) + '/' + os.path.basename(f).split('_')[0])
    docs = defaultdict(list)
    for f in files:
        f_name = os.path.dirname(f) + '/' + os.path.basename(f).split('_')[0]
        docs[f_name].append(f)
    return docs

In [329]:
docs = gather_docs(ocrs)

In [350]:
np.save('./all_docs.npy', data)

In [346]:
data = {}
for d, f in docs.items():
    line = ''
    for csv in f:
        try:
            _, rows = make_rows(pd.read_csv(csv))
        except:
            rows = ''
        add = ' '.join(rows)
        add += ' '
        line += add
    data[d] = line

In [1]:
import tensorflow_hub as hub


In [3]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

UNIVERSAL_ENCODER = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")



In [4]:
data = np.load('./all_docs.npy', allow_pickle=True).item()

In [5]:
from collections import defaultdict

In [18]:
features = dict.fromkeys(data)

In [20]:
from tqdm import tqdm

In [21]:
for k,v in tqdm(data.items()):
    features[k] = UNIVERSAL_ENCODER([v])

100%|██████████| 324/324 [00:41<00:00,  7.89it/s]


In [22]:
np.save('features.npy', features)

In [26]:
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans

ImportError: cannot import name 'KMeans'

In [6]:
one_e, two_e = UNIVERSAL_ENCODER([data['../data/google/1_БТИ/1. ул Шеногина, дом 3, строение 14 изм'], 
                                  data['../data/google/2_ЗУ/М-04-582527']])

corr = np.inner(one_e, two_e)

In [9]:
one_e.numpy().shape

(512,)

In [17]:
corr

0.3282603

In [ ]:
data.keys()

In [ ]:
data['../data/google/1_БТИ/1 изм']

In [ ]:
make_rows(pd.read_csv(docs['../data/google/1_БТИ/1 изм'][0]))[1]

In [ ]:
data = {}
for k,v in docs:
    data[k] = 

In [314]:
%%time
ocrs = list(sorted(ocrs))
res = (classify_docs(ocrs))
        

CPU times: user 45.4 s, sys: 492 ms, total: 45.9 s
Wall time: 46.7 s


In [ ]:
import collections

od = collections.OrderedDict(sorted(res.items()))


od = dict(od)

for k,v in od.items():
    if v is None:
        print(k)